# Importsand preparations

In [1]:
import pandas as pd
import torch
import os
import numpy as np
import datasets
import transformers
from GPUtil import showUtilization as gpu_usage
from numba import cuda
import torch.nn.functional as F

from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
)

from datasets import load_dataset, Dataset, DatasetDict

In [2]:
# !watch -n 0.5 nvidia-smi

In [3]:
print(f'PyTorch version: {torch.__version__}')  # 1.9.1+cu111
print(f'CUDA version: {torch.version.cuda}')  # 11.1
print(f'cuDNN version: {torch.backends.cudnn.version()}')  # 8005
print(f'Current device: {torch.cuda.current_device()}')  # 0
print(f'Is cuda available: {torch.cuda.is_available()}')  # TRUE

PyTorch version: 2.0.1+cu117
CUDA version: 11.7
cuDNN version: 8500
Current device: 0
Is cuda available: True


In [4]:
print(f'Transformers version: {transformers.__version__}')
print(f'Datasets version: {datasets.__version__}')

Transformers version: 4.32.1
Datasets version: 2.14.4


In [5]:
# Prevent a warning related to the tokenization process in the transformers library. 
os.environ["TOKENIZERS_PARALLELISM"] = "False"
# Makes CUDA operations synchronous
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [6]:
# Find the GPU with the least memory usage.
!nvidia-smi

Thu Oct 26 12:25:29 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti      On | 00000000:01:00.0 Off |                  N/A |
| 23%   43C    P2               62W / 250W|    949MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [7]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    # free unreferenced tensors from the GPU memory.
    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU  | MEM |
-------------------
|  0 |   0% |  8% |
|  1 | 100% | 82% |
|  2 | 100% | 82% |
|  3 | 100% | 82% |
|  4 |   0% | 12% |
|  5 |   0% |  0% |
|  6 |   0% |  0% |
|  7 | 100% | 82% |
GPU Usage after emptying the cache
| ID | GPU  | MEM |
-------------------
|  0 |   0% |  8% |
|  1 | 100% | 82% |
|  2 | 100% | 82% |
|  3 | 100% | 82% |
|  4 |   0% | 12% |
|  5 |   0% |  0% |
|  6 |   1% |  1% |
|  7 | 100% | 82% |


In [8]:
data = pd.read_csv("csv/priority_high_or_not_high_clean.csv" , index_col = 0)
data

,text_clean,label
0,create new vignette on project team grading wi...,0
1,app crashes when darryl visits messagesscreen ...,0
2,missing onrequeststart rolecheck dont always w...,0
3,describe the bug i noticed i now have the opti...,0
4,describe the bug h make hash 1 2 3 4 5 make ha...,0
...,...,...
112304,with alertlistings collections passed in to qu...,1
112305,for some reason the datetime in planner is not...,1
112306,hi all i found some mistakes in our document s...,1
112307,id 0x7b7d2f7b7d2f7b6578706f727465645f6e616d657...,1


In [9]:
# Smaller and farter than bert.
base_model_id = "distilbert-base-uncased"

epochs = 5 #Number of full cyles through the training set.
num_labels = 2 #Number of labels, high, med, low priority.
learning_rate = 5e-5 # Rate the model updates based on the data its trained on.
train_batch_size = 16 # Number of training examples in one iteration.
eval_batch_size = 32 # Number evalutaion examples in on iteratoion.
save_strategy = "no" # Should the model be saved automatically during training.
save_steps = 500 # How often to save the model during training. No effect since no over.
logging_steps = 100
model_dir = "./model" #Where to save model

# Use early stopping to prevent overfitting
#load_best_model_at_end=True
#metric_for_best_model="eval_loss"
#greater_is_better=False

In [10]:

# Split dataframe into three parts: training, validation and testing.
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [11]:
# Drops rows with missing values
data.dropna(inplace=True)

In [12]:
# Resets the index after dropping rows
data.reset_index(inplace=True)
data

,index,text_clean,label
0,0,create new vignette on project team grading wi...,0
1,1,app crashes when darryl visits messagesscreen ...,0
2,2,missing onrequeststart rolecheck dont always w...,0
3,3,describe the bug i noticed i now have the opti...,0
4,4,describe the bug h make hash 1 2 3 4 5 make ha...,0
...,...,...,...
112304,112304,with alertlistings collections passed in to qu...,1
112305,112305,for some reason the datetime in planner is not...,1
112306,112306,hi all i found some mistakes in our document s...,1
112307,112307,id 0x7b7d2f7b7d2f7b6578706f727465645f6e616d657...,1


In [13]:
#Drops the index col, better for managint the data.
data.drop(columns= ["index"], inplace = True)

In [14]:
data

,text_clean,label
0,create new vignette on project team grading wi...,0
1,app crashes when darryl visits messagesscreen ...,0
2,missing onrequeststart rolecheck dont always w...,0
3,describe the bug i noticed i now have the opti...,0
4,describe the bug h make hash 1 2 3 4 5 make ha...,0
...,...,...
112304,with alertlistings collections passed in to qu...,1
112305,for some reason the datetime in planner is not...,1
112306,hi all i found some mistakes in our document s...,1
112307,id 0x7b7d2f7b7d2f7b6578706f727465645f6e616d657...,1


In [15]:
# 60% trainig, 20% validate, 20% test. Seed None.
# Test 80-10-10 and 70-15-15
train , validate , test = train_validate_test_split(data)

In [16]:
train.set_index("label" , inplace = True)
validate.set_index("label" , inplace = True)
test.set_index("label" , inplace = True)

In [17]:
test

,text_clean
label,
0,describe the bug when using connectorcreate in...
0,the exporter appears to be randomly ordering t...
0,forum posthttpswwwbryntumcomforumviewtopicphpf...
0,if an incorrect formation is loaded or misspel...
1,section 3 talks about the challenges of 0rtt b...
...,...
0,using the quorum intersection checker provided...
1,cve202123337 high severity vulnerability detai...
1,background we need to ensure proper loading an...


In [18]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
testds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = testds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 22463
    })
    train: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 67385
    })
    validate: Dataset({
        features: ['text_clean', 'label'],
        num_rows: 22461
    })
})

In [19]:
train_dataset = ds["train"]
valid_dataset = ds["validate"]
test_ds = ds["test"]

In [20]:
ds["train"][0]

{'text_clean': 'thanks for ardi it is exactly what i was planning to write but already written  the little mini serial monitor in attachandwatch is great especially with how it integrates with the other attachandwatch functionality stopping during upload but it seems to assume 115200 baud thats a reasonable assumption to be sure but it seems worth adding some sort of configuration parameter maybe something that can end up in ardijson this is low priority since most sensible people use 115200 baud anyway its the fastest widely supported serial baud rate however it should probably be documented',
 'label': 1}

In [21]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(base_model_id, num_labels=num_labels)
#tokenizer = AutoTokenizer.from_pretrained(base_model_id)
# optim = torch.optim.Adam(model.parameters(), lr=5e-5)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Tokenization

In [23]:
model_ckpt = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

    Tokenizing the whole dataset

In [24]:
#Tokenize the dataset to the correct input for the transformer model.
def tokenize(batch):
    return tokenizer(batch["text_clean"], padding="max_length", truncation=True)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
valid_dataset = valid_dataset.map(tokenize, batched=True, batch_size=len(valid_dataset))
test_dataset = test_ds.map(tokenize, batched=True, batch_size=len(test_ds))

Map:   0%|          | 0/67385 [00:00<?, ? examples/s]

Map:   0%|          | 0/22461 [00:00<?, ? examples/s]

Map:   0%|          | 0/22463 [00:00<?, ? examples/s]

## Training a classifier

In [25]:
training_args = TrainingArguments(
    output_dir=model_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=eval_batch_size,
    save_strategy=save_strategy,
    save_steps=save_steps,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    logging_steps=logging_steps,
)

In [26]:
 trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
)

In [27]:
trainer.train() 

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.448300,0.452657,0.760963,[0.68906006 0.80585789],[0.85168218 0.72001809],[0.57858393 0.91493554]
2,0.394700,0.452800,0.770981,[0.76633052 0.77544962],[0.71897375 0.82784975],[0.82036569 0.72928812]
3,0.262000,0.551925,0.779484,[0.74841266 0.80372499],[0.78330675 0.776731 ],[0.71649485 0.83266278]
4,0.175500,0.919881,0.779573,[0.75939155 0.79663175],[0.75891209 0.79705737],[0.75987162 0.79620659]
5,0.102700,1.183411,0.777214,[0.75879688 0.79301787],[0.75219801 0.79903309],[0.76551255 0.78709254]


TrainOutput(global_step=21060, training_loss=0.2807473910160554, metrics={'train_runtime': 7186.3586, 'train_samples_per_second': 46.884, 'train_steps_per_second': 2.931, 'total_flos': 4.46315782921728e+16, 'train_loss': 0.2807473910160554, 'epoch': 5.0})

* Training loss: Difference between the predictons made by the model on the training dataset vs on the actual data.
* Validation loss: how well the model functions on unseen data.
* Accuracy: How much the model gets correct. number of correct Prediction / total number of predictions.
* F1: consider both precision and recall. 
* Precision: Accuracy of positive predictions. Percison TP = TP + FP. How often the model is correct.
* Recall: True positive rate. how many items the model gets correct from the total amount.

### Training loss decreases, valdiation loss increases = Overfitting

In [28]:
# Evaluate valdiation set
eval_result = trainer.evaluate(eval_dataset=valid_dataset)

In [29]:
for key, value in sorted(eval_result.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.7772138373180179

eval_f1 = [0.75879688 0.79301787]

eval_loss = 1.1834113597869873

eval_precision = [0.75219801 0.79903309]

eval_recall = [0.76551255 0.78709254]

eval_runtime = 144.5051

eval_samples_per_second = 155.434

eval_steps_per_second = 4.858



In [30]:
# Evaluate test data set
test_results = trainer.evaluate(eval_dataset=test_dataset)

In [31]:
for key, value in sorted(test_results.items()):
    print(f"{key} = {value}\n")

epoch = 5.0

eval_accuracy = 0.7770110848951609

eval_f1 = [0.76259538 0.7897763 ]

eval_loss = 1.184050440788269

eval_precision = [0.76219801 0.79014108]

eval_recall = [0.76299317 0.78941186]

eval_runtime = 144.5711

eval_samples_per_second = 155.377

eval_steps_per_second = 4.856



In [32]:
trainer.save_model(model_dir + "_local") 

In [33]:
from transformers import pipeline
    
classifier = pipeline("text-classification", model="./model_local")

In [34]:
classifier.model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [35]:
classifier("this does not need to be done fast")

[{'label': 'LABEL_1', 'score': 0.9585654139518738}]

In [41]:
classifier("this is super important")

[{'label': 'LABEL_0', 'score': 0.9984287619590759}]

In [43]:
classifier("this bug has super high impact on the project")

[{'label': 'LABEL_0', 'score': 0.9995484948158264}]

## Important to delete large objects to free memory 
del train_dataset

In [36]:
del valid_dataset

In [37]:
del model

In [38]:
# Free cache
torch.cuda.empty_cache()

In [39]:
!nvidia-smi

Thu Oct 26 14:32:59 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti      On | 00000000:01:00.0 Off |                  N/A |
| 23%   42C    P2               62W / 250W|    949MiB / 11264MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--